In [1]:
# Calculate document vectors (average of word vectors)
def average_word_vectors(words, model, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    n_words = 0

    for word in words:
        if word in model.wv:
            n_words += 1
            feature_vector = np.add(feature_vector, model.wv[word])

    if n_words > 0:
        feature_vector = np.divide(feature_vector, n_words)
    
    return feature_vector


In [4]:
!pip install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [16]:
import numpy as np
from gensim.models import Word2Vec

# Sample user profiles and project descriptions
user_profiles = [
    {
        'mandatory_skills': ['data', 'science', 'machine', 'learning','software engineering'],
        'good_to_have_skills': ['statistics', 'python', 'programming'],
        'expertise_level': 0.8,
        'contributor_type': 'developer'  # Example contributor type
    },
    {
        'mandatory_skills': ['biology', 'research', 'genetics', 'lab'],
        'good_to_have_skills': ['microbiology', 'DNA sequencing'],
        'expertise_level': 0.6,
        'contributor_type': 'researcher'
    },
    {
        'mandatory_skills': ['environment', 'sustainability', 'climate', 'change'],
        'good_to_have_skills': ['renewable energy', 'carbon footprint'],
        'expertise_level': 0.7,
        'contributor_type': 'environmentalist'
    },
]

project_descriptions = [
    {
        'mandatory_skills': ['data', 'science', 'machine', 'learning'],
        'good_to_have_skills': ['machine learning', 'data visualization'],
        'expertise_level': 0.75,
        'contributor_type': 'developer'
    },
    {
        'mandatory_skills': ['genomics', 'study', 'gene', 'expression'],
        'good_to_have_skills': ['genome sequencing', 'biological research'],
        'expertise_level': 0.65,
        'contributor_type': 'researcher'
    },
    {
        'mandatory_skills': ['sustainable', 'solutions', 'carbon', 'offset'],
        'good_to_have_skills': ['renewable energy', 'environmental policy'],
        'expertise_level': 0.72,
        'contributor_type': 'environmentalist'
    },
]

# Train Word2Vec model
model = Word2Vec([profile['mandatory_skills'] + profile['good_to_have_skills'] for profile in user_profiles + project_descriptions], vector_size=100, window=5, min_count=1, sg=0)

# Function to calculate cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Threshold for considering expertise level similarity
similarity_threshold = 0.3

# Iterate through user profiles and project descriptions
for i, user_profile in enumerate(user_profiles):
    for j, project_description in enumerate(project_descriptions):
        # Check if the contributor types match
        if user_profile['contributor_type'] == project_description['contributor_type']:
            # Calculate similarity for mandatory skills
            mandatory_skills_similarity = cosine_similarity(average_word_vectors(user_profile['mandatory_skills'], model, 100), average_word_vectors(project_description['mandatory_skills'], model, 100))
            
            # Calculate similarity for good-to-have skills
            good_to_have_skills_similarity = cosine_similarity(average_word_vectors(user_profile['good_to_have_skills'], model, 100), average_word_vectors(project_description['good_to_have_skills'], model, 100))
            
            # Check if either of the skills similarities exceeds the threshold
            if mandatory_skills_similarity > similarity_threshold or good_to_have_skills_similarity > similarity_threshold:
                # Calculate similarity for expertise level (you can adjust the weight as needed)
                expertise_level_similarity = 1 - abs(user_profile['expertise_level'] - project_description['expertise_level'])
                
                # Print the similarities for each aspect
                print(f"User Profile {i+1} vs. Project {j+1}:")
                print(f"Mandatory Skills Similarity: {mandatory_skills_similarity:.2f}")
                print(f"Good-to-Have Skills Similarity: {good_to_have_skills_similarity:.2f}")
                
                if expertise_level_similarity is not None:
                    # Print the expertise level similarity
                    print(f"Expertise Level Similarity: {expertise_level_similarity:.2f}")
                    
                    # Sum up the similarities for all aspects
                    combined_similarity = mandatory_skills_similarity + good_to_have_skills_similarity + expertise_level_similarity
                    # Print the combined similarity
                    print(f"Combined Similarity: {combined_similarity:.2f}")
                else:
                    print("Expertise level similarity not considered.")
                    
                print()  # Add a newline for clarity
            else:
                print(f"User Profile {i+1} vs. Project {j+1}:")
                print("Skills similarity below threshold, expertise level similarity not considered.")
                print()
        else:
            print(f"User Profile {i+1} vs. Project {j+1}:")
            print("Contributor types do not match, skipping other similarities.")
            print()
# Define a custom JSON encoder to handle non-serializable types
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()  # Convert NumPy arrays to Python lists
        return super().default(obj)

# Initialize a list to store the results
results = []

# Iterate through user profiles and project descriptions
for i, user_profile in enumerate(user_profiles):
    for j, project_description in enumerate(project_descriptions):
        # Check if the contributor types match
        if user_profile['contributor_type'] == project_description['contributor_type']:
            # ... (your similarity calculations)

            # Convert NumPy arrays to Python lists within the result dictionary
            for key, value in result.items():
                if isinstance(value, np.ndarray):
                    result[key] = value.tolist()
                elif isinstance(value, np.float32):
                    result[key] = float(value)  # Convert np.float32 to native Python float

            # Append the result to the list
            results.append(result)

# Save the results to a JSON file using the custom JSON encoder
with open('similarities1.json', 'w') as json_file:
    json.dump(results, json_file, indent=4, cls=CustomJSONEncoder)


User Profile 1 vs. Project 1:
Mandatory Skills Similarity: 0.87
Good-to-Have Skills Similarity: 0.07
Expertise Level Similarity: 0.95
Combined Similarity: 1.89

User Profile 1 vs. Project 2:
Contributor types do not match, skipping other similarities.

User Profile 1 vs. Project 3:
Contributor types do not match, skipping other similarities.

User Profile 2 vs. Project 1:
Contributor types do not match, skipping other similarities.

User Profile 2 vs. Project 2:
Skills similarity below threshold, expertise level similarity not considered.

User Profile 2 vs. Project 3:
Contributor types do not match, skipping other similarities.

User Profile 3 vs. Project 1:
Contributor types do not match, skipping other similarities.

User Profile 3 vs. Project 2:
Contributor types do not match, skipping other similarities.

User Profile 3 vs. Project 3:
Mandatory Skills Similarity: -0.11
Good-to-Have Skills Similarity: 0.51
Expertise Level Similarity: 0.98
Combined Similarity: 1.37

